# 「Python による経済・経営分析のための実践的データサイエンス」第6章 6.8, 6.9 (pp.206-219)

## 6.8 因果推論(1) 傾向スコアマッチング(PSM)



In [ ]:
import pandas as pd
input_book = pd.ExcelFile('FIFA19_data.xlsx')

input_sheet_name = input_book.sheet_names
num_sheet = len(input_sheet_name)
print(input_sheet_name)
print("Sheet の数:", num_sheet)
input_sheet_df = input_book.parse(input_sheet_name[0])

#GK のデータのみを外す
input_sheet_df = input_sheet_df[input_sheet_df['Position'] != "GK"]
#この箇所をコメントアウトすると、ゴールキーパーを含んだ形での分析になります

#最初の10行のみを表示する
input_sheet_df.head(10)

['data', 'Sheet1']
Sheet の数: 2


,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,fifarank,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,11,FC Barcelona,...,96,33,28,26,6,11,15,14,8,€226.5M
1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,7,Juventus,...,95,28,31,23,7,11,15,14,11,€127.1M
2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,3,Paris Saint-Germain,...,94,27,24,33,9,9,15,15,11,€228.1M
4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,1,Manchester City,...,88,68,58,51,15,13,5,10,13,€196.4M
5,183277,E. Hazard,27,https://cdn.sofifa.org/players/4/19/183277.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,91,1,Chelsea,...,91,34,27,22,11,12,6,8,8,€172.1M
6,177003,L. Modrić,32,https://cdn.sofifa.org/players/4/19/177003.png,Croatia,https://cdn.sofifa.org/flags/10.png,91,91,5,Real Madrid,...,84,60,76,73,13,9,7,14,9,€137.4M
7,176580,L. Suárez,31,https://cdn.sofifa.org/players/4/19/176580.png,Uruguay,https://cdn.sofifa.org/flags/60.png,91,91,6,FC Barcelona,...,85,62,45,38,27,25,31,33,37,€164M
8,155862,Sergio Ramos,32,https://cdn.sofifa.org/players/4/19/155862.png,Spain,https://cdn.sofifa.org/flags/45.png,91,91,9,Real Madrid,...,82,87,92,91,11,8,9,7,11,€104.6M
10,188545,R. Lewandowski,29,https://cdn.sofifa.org/players/4/19/188545.png,Poland,https://cdn.sofifa.org/flags/37.png,90,90,20,FC Bayern München,...,86,34,42,19,15,6,12,8,10,€127.1M
11,182521,T. Kroos,28,https://cdn.sofifa.org/players/4/19/182521.png,Germany,https://cdn.sofifa.org/flags/21.png,90,90,13,Real Madrid,...,85,72,79,69,10,11,13,7,10,€156.8M


In [ ]:
import numpy as np

#データを読み込む
age=input_sheet_df.Age #年齢
overall=input_sheet_df.Overall #総合能力
wage=input_sheet_df.Wage2 #給与
preferredfoot=input_sheet_df.preferredfoot #利き足
reputation=input_sheet_df.reputation #レピュテーション
least_contract=input_sheet_df.least_contract #残りの契約年数
height=input_sheet_df.height2 #身長
weight=input_sheet_df.weight2 #体重
crossing=input_sheet_df.Crossing #クロス精度
finishing=input_sheet_df.Finishing #フィニッシュ精度
heading=input_sheet_df.HeadingAccuracy #ヘディング精度
shortPassing=input_sheet_df.ShortPassing #ショートパス精度
dribbling=input_sheet_df.Dribbling #ドリブルの精度
Curve=input_sheet_df.Curve #カーブの精度
FKAccuracy=input_sheet_df.FKAccuracy #FK の精度
LongPassing=input_sheet_df.LongPassing #ロングパスの精度
BallControl=input_sheet_df.BallControl #ボールコントロール
Acceleration=input_sheet_df.Acceleration #飛び出し
SprintSpeed=input_sheet_df.SprintSpeed #スプリントスピード
Agility=input_sheet_df.Agility #アジリティ
Reactions=input_sheet_df.Reactions #リアクション
Balance=input_sheet_df.Balance #バランス
ShotPower=input_sheet_df.ShotPower #シュートパワー
stamina=input_sheet_df.Stamina #スタミナ
Jumping=input_sheet_df.Jumping #ジャンプ
Strength=input_sheet_df.Strength #ストレングス
LongShots=input_sheet_df.LongShots #ロングスート
Aggression=input_sheet_df.Aggression #アグレッション
Interceptions=input_sheet_df.Interceptions #インターセプト
Positioning=input_sheet_df.Positioning #ポジショニング
Vision=input_sheet_df.Vision #ビジョン
Penalties=input_sheet_df.Penalties #ペナルティキック
Composure=input_sheet_df.Composure #コンポジュア
Marking=input_sheet_df.Marking #マーキング
StandingTackle=input_sheet_df.StandingTackle #スタンディングタックル
SlidingTackle=input_sheet_df.SlidingTackle #スライディングタックル

#利用するパラメータを指定する
#overall はここから除外しています。結果が変わるので、入れたバージョンも実施してみてください
equation_df3=pd.concat([wage, age, preferredfoot, reputation, least_contract, overall, finishing, \
dribbling, height, weight, crossing, finishing, heading, heading, shortPassing, Curve, FKAccuracy, \
LongPassing, BallControl, Acceleration, SprintSpeed, Agility, Reactions, \
Balance, ShotPower, stamina, Jumping, stamina, Strength, LongShots, Aggression, \
Interceptions, Positioning, Vision, Penalties, Composure, Marking, \
StandingTackle, SlidingTackle], axis=1)

#被説明変数として利用するものを取り出す
foot = pd.DataFrame(equation_df3.preferredfoot)
#被説明変数を抜き取る
x_list3 = equation_df3.drop("preferredfoot",axis=1)

#被説明変数として利用するものを取り出す
reputation = pd.DataFrame(equation_df3.reputation)
#被説明変数を抜き取る
x_list3 = x_list3.drop("reputation",axis=1)


#異常値やnull になっている値を除去する
x_list3 = x_list3.drop(x_list3.columns[np.isnan(x_list3).any()], axis=1)

In [8]:
import statsmodels.api as sm
#傾向スコアを求める
glm = sm.Logit(foot, x_list3)
#google colabで実施する場合、methodを指定しないとエラーとなるため、
#method='bfgs'(最尤法のアルゴリズム)を指定する。
result = glm.fit(method='bfgs')
#x_list3による解析の結果をpsに放り込む
ps = pd.Series(result.predict(x_list3))
#傾向スコアの値を確認する
print(ps)

         Current function value: 0.518382
         Iterations: 35
         Function evaluations: 50
         Gradient evaluations: 50
0        0.190274
1        0.138709
2        0.299577
4        0.246443
5        0.161163
           ...   
17913    0.137581
17914    0.113642
17915    0.113257
17916    0.274175
17917    0.214006
Length: 15926, dtype: float64


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:1292: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:595: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [ ]:
#Propensity score matching
table = pd.concat([ps,foot,wage],axis=1)
table.columns = ['ps','foot','wage']
print(table)

             ps  foot  wage
0      0.190274     1   565
1      0.138709     0   405
2      0.299577     0   290
4      0.246443     0   355
5      0.161163     0   340
...         ...   ...   ...
17913  0.137581     0     1
17914  0.113642     0     1
17915  0.113257     0     1
17916  0.274175     0     1
17917  0.214006     0     1

[15926 rows x 3 columns]


In [ ]:
interval = np.arange(0,1.001,0.01)
match_list = []
for i in range(0,len(interval)-1):
    temp0 = table[(table['foot']==0) & (interval[i] < table['ps']) & (table['ps'] < interval[i+1])]
    temp1 = table[(table['foot']==1) & (interval[i] < table['ps']) & (table['ps'] < interval[i+1])]
    if (len(temp0) > 0) & (len(temp1) > 0):
        match_list.append(temp1['wage'].mean()-temp0['wage'].mean())

np.mean(match_list)

-0.47359465303315307